In [ ]:
import rasterio  
import rasterio.mask
import geopandas as gps
import os
import time
import shutil
from shapely.geometry import Point, Polygon,box
import numpy as np

In [ ]:
auxiliary_data_dir=r'D:\lakemapping\0_auxiliary_data/auxiliary_dataset.gdb'
# grwl_path = os.path.join(auxiliary_data_dir, 'GRWL') 
# OSM_path =  os.path.join(auxiliary_data_dir, 'OSM_water') 
raw_prediction_dir=r'F:\result_0220\tif'
output_dir=r'F:\result_0220\after_land_mask'

In [ ]:
land_mask = gps.read_file(auxiliary_data_dir,layer='continent_polygon')
land_mask

## 1 land mask

In [ ]:
start_id=6001
end_id=start_id+1000
out_dir=os.path.join(output_dir,'{}_{}'.format(start_id,end_id-1))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
start=time.time()
for index in range(6826,end_id):
# for index in range(9294,9295):    
    raw_pre_dir=os.path.join(raw_prediction_dir,'{}_{}/pre_{}_iew100.tif'.format(start_id,end_id-1,index))
    out_pre_dir=os.path.join(out_dir,'pre_{}_alm.tif'.format(index))
#     print(out_pre_dir)
    raw_prediction_image=rasterio.open(raw_pre_dir)
#     print(raw_prediction_image)
    bbox = box(*raw_prediction_image.bounds)
    meta=raw_prediction_image.meta
    meta.update({"compress": 'lzw'})
    # grwl_river_patch = gps.read_file(auxiliary_data_dir,layer='GRWL_DNRiver' ,bbox=bbox) # data from GRWL_mask_V01.01 product with DN = 255 (River) (see https://doi.org/10.5281/zenodo.1297434).
    # ocean_patch = gps.read_file(auxiliary_data_dir,layer='oceanline_polygon', bbox=bbox) 
    # mask_patch = ocean_patch#grwl_river_patch._append(ocean_patch)
    # mask_patch_geojson = mask_patch.geometry.values 
    land_mask = gps.read_file(r'D:\lakemapping\8_postprocess\region\continent_polygon.shp', bbox=bbox) 
    mask_patch_geojson = land_mask.geometry.values 
 # perform land mask
    if len(mask_patch_geojson) > 0:
        modelpredtictions_masked, out_meta = rasterio.mask.mask(raw_prediction_image, mask_patch_geojson, all_touched=True, invert=False)
        print(f'Mask finished: {index} / {end_id-1}')
        with rasterio.open(out_pre_dir, "w", **meta) as prediction_masked:
            prediction_masked.write(modelpredtictions_masked)
    
end=time.time()
print(f'time:{end-start}')

## classify to different region

In [ ]:
continent_polygon = gps.read_file(auxiliary_data_dir,layer='continent_polygon')
continent_polygon

In [ ]:
region_List=continent_polygon['region'].unique().tolist()

In [ ]:
for region in region_List:
    os.makedirs(r'D:\postprocess\v240220\region\{}'.format(region))

In [ ]:
for region in region_List:
    region_clip=continent_polygon[continent_polygon.region==region]
    id_list=region_clip.Id.tolist()
    for i in id_list:
        for root, dirs, files in os.walk(r'D:\postprocess\v240220\shp_dn1'):
            for file in files:
                if file.startswith(f'pre_{i}_alm2shp_dn1') or file.startswith(f'pre_{i}_after_land_mask2shp_dn1'):
                    old_file=os.path.join(root, file)
                    new_file=os.path.join(r'D:\postprocess\v240220\region',region+'/'+file)
#                     print(old_file)
#                     print(new_file)
                    shutil.move(old_file,new_file)
                    continue